In [2]:
import torch
import numpy as np
# import torchvision
# from torchvision.transforms import ToTensor
# from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        #define Model Layers here:
        
        
        ###
    
    def forward(self, x):
        #define Forward pass here:
        
        
        ###
        return y
    
    def training_step(self, batch, loss_fn):
        x, y = batch
        y_pred = self(x)
        loss = loss_fn(y_pred, y)
        return loss
    
    def validation_step(self, batch, loss_fn, metrics=None):
        x, y = batch
        y_pred = self(x)
        loss = loss_fn(y_pred, y)
        
        metric = None
        if metrics is not None:
            metric = metrics(y_pred, y)
        
        return {'val_loss' : loss, f'val_{metrics.__name__}' : metric}
    
    def validation_epoch_end(self, val_dl, loss_fn,  metrics = None):
        val_batches = [self.validation_step(batch, loss_fn, metrics) for batch in val_dl]
        
        #Epoch Loss
        batch_losses = [x['val_loss'] for x in val_batches]
        epoch_loss = torch.stack(batch_losses).mean()
        print('val_loss: ', epoch_loss.item())
        
        #Metrics
        if metrics is not None:
            batch_mets = [x[f'val_{metrics.__name__}'] for x in val_batches]
            epoch_mets = torch.stack(batch_mets).mean() 
            print(f'val_{metrics.__name__}: ', epoch_mets.item())
            
        return (epoch_loss.item(), epoch_mets.item())
            
    def epoch_end(self, epoch, val_dl, loss_fn, metrics=None):
        print(f'Epoch {epoch+1}:')
        return self.validation_epoch_end(val_dl, loss_fn, metrics)
        
    def fit(self, train_dl, val_dl, epochs, lr, loss_fn, opt_fn = torch.optim.SGD, metrics=None):
        history = []
        opt = opt_fn(self.parameters(), lr=lr)
        for epoch in range(epochs):
            # Training Phase
            for batch in train_dl:
                loss = self.training_step(batch, loss_fn)
                loss.backward()
                opt.step()
                opt.zero_grad()
                
            # Validation Phase
            history.append(self.epoch_end(epoch, val_dl, loss_fn, metrics))
        return history
        

In [4]:
# Accuracy metrics For MultiClass Classification
def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == labels) / len(labels))